# Ensembles using Bootstrap Aggregration
# Basic Steps
# 1) Randomly choosing the samples from the original dataset.
# 2) Model is been created from each Bootstrap sample from the original dataset WITH REPLACEMENT.
# 3)Now each model is been trained indepedently which generates the results.
# 4) All the outputs/results generated by each model is been combined using majority voting technique and this step is called aggregation.

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [5]:
data = pd.read_csv('heart_disease_data.csv')

In [29]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slop,ca,thal,pred_attribute
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [6]:
X = data.drop('pred_attribute', axis=1)
y = data['pred_attribute']

In [7]:
X = X.fillna(X.mean())  # Replace missing values with the mean of each column

C:\Users\suzen\AppData\Local\Temp\ipykernel_9672\154075794.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X = X.fillna(X.mean())  # Replace missing values with the mean of each column


In [8]:
# Assuming 'sex', 'cp', 'fbs', 'restecg', 'exang', 'slop', 'ca', 'thal' are categorical columns

cat_columns = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slop', 'ca', 'thal']
encoder = LabelEncoder()

for column in cat_columns:
    X[column] = encoder.fit_transform(X[column])


In [9]:
# Assuming 'age', 'trestbps', 'chol', 'thalach', 'oldpeak' are numerical columns

num_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
scaler = StandardScaler()

X[num_columns] = scaler.fit_transform(X[num_columns])


In [18]:
data.isnull().sum()

age               0
sex               0
cp                0
trestbps          0
chol              0
fbs               0
restecg           0
thalach           0
exang             0
oldpeak           0
slop              0
ca                0
thal              0
pred_attribute    0
dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create a base classifier (e.g., Decision Tree)
base_classifier = DecisionTreeClassifier()


In [23]:
# Create the bagging classifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

In [24]:
# Create the voting classifier with majority voting
voting_classifier = VotingClassifier(estimators=[('bagging', bagging_classifier)], voting='hard')

In [25]:
# Train the voting classifier using the training data
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('bagging',
                              BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                                random_state=42))])

In [26]:
# Predict the target variable for the test data
y_pred = voting_classifier.predict(X_test)

In [27]:
# Calculate the accuracy of the voting classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5409836065573771
